In [6]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from datetime import datetime
from typing import List, Tuple, Dict, Any, Optional, Union
from dataclasses import dataclass, field
import logging

# ML imports
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.feature_selection import SelectFromModel
import joblib

# Suppress warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')


# Configuration & Setup

@dataclass
class ROIEngineConfig:
    """Advanced configuration for ROI Engine"""
    # Data paths
    base_dir: str = "./roi_engine_project"
    data_filename: str = "synthetic_cre_dataset.csv"

    # Model settings
    target: str = "total_return"
    date_col: str = "date"
    id_cols: List[str] = field(default_factory=lambda: ["market", "asset_type"])

    # Cross-validation
    n_splits: int = 5
    test_size_ratio: float = 0.2

    # Model parameters
    random_state: int = 42
    n_jobs: int = -1

    # Feature engineering
    enable_advanced_features: bool = True
    rolling_windows: List[int] = field(default_factory=lambda: [3, 6, 12])
    lag_periods: List[int] = field(default_factory=lambda: [1, 2, 3, 6])

    # Model selection
    models_to_try: List[str] = field(default_factory=lambda: ['gradient_boosting', 'random_forest', 'ridge'])
    enable_hyperparameter_tuning: bool = True

    # Output settings
    save_plots: bool = True
    plot_format: str = "png"
    plot_dpi: int = 300

    def __post_init__(self):
        self.data_dir = Path(self.base_dir) / "data"
        self.artifacts_dir = Path(self.base_dir) / "artifacts"
        self.plots_dir = self.artifacts_dir / "plots"

        # Create directories
        for dir_path in [self.data_dir, self.artifacts_dir, self.plots_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)

        self.data_path = self.data_dir / self.data_filename


# Logging Setup

def setup_logging(config: ROIEngineConfig) -> logging.Logger:
    """Setup logging configuration"""
    log_file = config.artifacts_dir / "roi_engine.log"

    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )

    logger = logging.getLogger('ROI_Engine')
    return logger


# Enhanced Data Generation

class SyntheticDataGenerator:
    """Enhanced synthetic CRE data generator with realistic market dynamics"""

    def __init__(self, config: ROIEngineConfig, logger: logging.Logger):
        self.config = config
        self.logger = logger

    def generate_macroeconomic_data(self, n_periods: int, start_date: str = "2016-01-01") -> pd.DataFrame:
        """Generate realistic macroeconomic time series"""
        np.random.seed(self.config.random_state)

        dates = pd.date_range(start_date, periods=n_periods, freq="M")
        time_trend = np.linspace(0, 8, n_periods)

        # More realistic macro indicators with autocorrelation
        inflation_base = 2.0 + 0.8 * np.sin(time_trend) + np.random.normal(0, 0.3, n_periods)
        inflation = np.cumsum(np.random.normal(0, 0.1, n_periods)) + inflation_base
        inflation = np.clip(inflation, 0.5, 6.0)

        fed_rate_changes = np.random.normal(0, 0.05, n_periods)
        fed_rate_changes[::24] += np.random.normal(0, 0.3, len(fed_rate_changes[::24]))  # Policy changes
        fed_rate = np.clip(np.cumsum(fed_rate_changes) + 1.5, 0.1, 8.0)

        gdp_base = 2.5 + 0.5 * np.cos(time_trend * 1.2)
        gdp_shocks = np.random.choice([0, 0, 0, 0, -2, 1.5], n_periods, p=[0.7, 0.1, 0.1, 0.05, 0.03, 0.02])
        gdp_growth = gdp_base + gdp_shocks + np.random.normal(0, 0.2, n_periods)
        gdp_growth = np.clip(gdp_growth, -3.0, 6.0)

        # Employment and credit conditions
        unemployment = np.clip(6.0 - 0.3 * gdp_growth + np.random.normal(0, 0.3, n_periods), 3.0, 12.0)
        credit_spread = np.clip(1.5 + 0.2 * (fed_rate - 2.0) - 0.1 * gdp_growth +
                               np.random.normal(0, 0.2, n_periods), 0.5, 4.0)

        return pd.DataFrame({
            'date': dates,
            'inflation': inflation,
            'fed_rate': fed_rate,
            'gdp_growth': gdp_growth,
            'unemployment': unemployment,
            'credit_spread': credit_spread
        })

    def generate_market_data(self, macro_df: pd.DataFrame) -> pd.DataFrame:
        """Generate market-specific real estate data"""
        n_periods = len(macro_df)

        # Market definitions with more granular characteristics
        market_params = {
            "NYC": {"base_rent": 85, "volatility": 0.15, "liquidity": 0.9, "barrier_to_entry": 0.8},
            "LA": {"base_rent": 55, "volatility": 0.18, "liquidity": 0.7, "barrier_to_entry": 0.6},
            "SF": {"base_rent": 75, "volatility": 0.22, "liquidity": 0.6, "barrier_to_entry": 0.9},
            "CHI": {"base_rent": 40, "volatility": 0.12, "liquidity": 0.8, "barrier_to_entry": 0.4},
            "DAL": {"base_rent": 35, "volatility": 0.16, "liquidity": 0.7, "barrier_to_entry": 0.3},
            "BOS": {"base_rent": 65, "volatility": 0.14, "liquidity": 0.6, "barrier_to_entry": 0.7},
            "SEA": {"base_rent": 60, "volatility": 0.20, "liquidity": 0.5, "barrier_to_entry": 0.6}
        }

        asset_type_params = {
            "Multifamily": {"alpha": 1.15, "beta_gdp": 0.3, "beta_unemployment": -0.4, "cyclicality": 0.7},
            "Office": {"alpha": 1.0, "beta_gdp": 0.5, "beta_unemployment": -0.6, "cyclicality": 1.2},
            "Industrial": {"alpha": 1.05, "beta_gdp": 0.4, "beta_unemployment": -0.2, "cyclicality": 0.8},
            "Retail": {"alpha": 0.95, "beta_gdp": 0.6, "beta_unemployment": -0.8, "cyclicality": 1.5},
            "Hotel": {"alpha": 1.3, "beta_gdp": 0.8, "beta_unemployment": -1.0, "cyclicality": 2.0}
        }

        records = []

        for _, row in macro_df.iterrows():
            # Generate multiple properties per period for more data
            n_properties = np.random.randint(8, 15)

            for _ in range(n_properties):
                market = np.random.choice(list(market_params.keys()))
                asset_type = np.random.choice(list(asset_type_params.keys()))

                mp = market_params[market]
                atp = asset_type_params[asset_type]

                # Property characteristics
                sqft = np.random.randint(25_000, 500_000)
                age = np.random.randint(1, 50)
                quality_score = np.random.uniform(0.6, 1.0)

                # Dynamic rent calculation
                base_rent = mp["base_rent"] * atp["alpha"] * quality_score

                # Economic sensitivity
                gdp_effect = atp["beta_gdp"] * (row["gdp_growth"] - 2.5) / 2.5
                unemployment_effect = atp["beta_unemployment"] * (row["unemployment"] - 6.0) / 6.0

                market_volatility = mp["volatility"] * atp["cyclicality"]
                rent_shock = np.random.normal(0, market_volatility)

                rent_psf = base_rent * (1 + gdp_effect + unemployment_effect + rent_shock)
                rent_psf = np.clip(rent_psf, base_rent * 0.5, base_rent * 2.0)

                annual_rent = rent_psf * sqft

                # Vacancy dynamics
                base_vacancy = 5.0 + np.random.uniform(-1, 3)
                vacancy_sensitivity = -unemployment_effect * 2 + (row["fed_rate"] - 2.0) * 0.5
                vacancy_rate = np.clip(base_vacancy + vacancy_sensitivity +
                                     np.random.normal(0, 1.5), 1.0, 25.0)

                effective_rent = annual_rent * (1 - vacancy_rate / 100.0)

                # Operating expenses with inflation sensitivity
                opex_base = 0.25 + 0.05 * np.random.rand()
                opex_inflation = 0.5 * (row["inflation"] - 2.0) / 2.0
                opex_rate = np.clip(opex_base + opex_inflation, 0.15, 0.45)
                opex = effective_rent * opex_rate

                noi = effective_rent - opex

                # Cap rates with risk premiums
                base_cap = 4.5 + 0.5 * (1 - mp["liquidity"]) + 0.3 * atp["cyclicality"]
                risk_premium = 0.3 * (row["fed_rate"] - 2.0) + 0.2 * row["credit_spread"]
                market_risk = -0.1 * (row["gdp_growth"] - 2.5) / 2.5

                cap_rate = np.clip(base_cap + risk_premium + market_risk +
                                  np.random.normal(0, 0.2), 3.0, 12.0)

                value = noi / (cap_rate / 100.0) if cap_rate > 0 else 0

                # Financing
                ltv_base = 0.65
                ltv_adjustment = -0.05 * (row["fed_rate"] - 2.0) / 2.0 - 0.03 * row["credit_spread"]
                ltv = np.clip(ltv_base + ltv_adjustment + np.random.normal(0, 0.05), 0.4, 0.8)

                loan_amt = value * ltv
                coupon = (row["fed_rate"] + row["credit_spread"] + 1.5 +
                         0.5 * atp["cyclicality"] + np.random.normal(0, 0.3)) / 100.0
                coupon = np.clip(coupon, 0.02, 0.12)

                annual_debt_service = loan_amt * coupon

                # Returns calculation with market dynamics
                market_appreciation = (0.02 + 0.5 * (row["gdp_growth"] - 2.5) / 2.5 -
                                     0.3 * (row["fed_rate"] - 2.0) / 2.0)
                asset_specific = atp["beta_gdp"] * (row["gdp_growth"] - 2.5) / 10.0
                idiosyncratic = np.random.normal(0, market_volatility * 0.5)

                appreciation_rate = market_appreciation + asset_specific + idiosyncratic
                value_next = value * (1 + appreciation_rate)

                equity = np.maximum(value - loan_amt, 1e-6)
                cash_flow = noi - annual_debt_service
                total_return = (cash_flow + (value_next - value)) / equity

                # Additional metrics
                dscr = noi / annual_debt_service if annual_debt_service > 0 else np.inf
                cash_on_cash = cash_flow / equity

                records.append({
                    "date": row["date"],
                    "market": market,
                    "asset_type": asset_type,
                    "sqft": sqft,
                    "age": age,
                    "quality_score": quality_score,
                    "rent_psf": rent_psf,
                    "annual_rent": annual_rent,
                    "vacancy_rate": vacancy_rate,
                    "effective_rent": effective_rent,
                    "opex_rate": opex_rate,
                    "opex": opex,
                    "noi": noi,
                    "cap_rate": cap_rate,
                    "value": value,
                    "ltv": ltv,
                    "loan_amt": loan_amt,
                    "coupon": coupon * 100.0,
                    "annual_debt_service": annual_debt_service,
                    "equity": equity,
                    "value_next": value_next,
                    "total_return": total_return,
                    "dscr": dscr,
                    "cash_on_cash": cash_on_cash,
                    **{k: row[k] for k in ["inflation", "fed_rate", "gdp_growth", "unemployment", "credit_spread"]}
                })

        return pd.DataFrame(records)

    def generate_dataset(self, n_periods: int = 120) -> pd.DataFrame:
        """Generate complete synthetic dataset"""
        self.logger.info(f"Generating synthetic dataset with {n_periods} periods")

        # Generate macroeconomic data
        macro_df = self.generate_macroeconomic_data(n_periods)

        # Generate market data
        market_df = self.generate_market_data(macro_df)

        self.logger.info(f"Generated {len(market_df)} property records")
        return market_df


# Advanced Feature Engineering

class FeatureEngineer:
    """Advanced feature engineering for real estate data"""

    def __init__(self, config: ROIEngineConfig, logger: logging.Logger):
        self.config = config
        self.logger = logger

    def create_time_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create comprehensive time-based features"""
        df = df.copy()

        # Basic time features
        df["year"] = df[self.config.date_col].dt.year
        df["month"] = df[self.config.date_col].dt.month
        df["quarter"] = df[self.config.date_col].dt.quarter
        df["year_month"] = df[self.config.date_col].dt.to_period('M').astype(str)

        # Cyclical encoding for seasonal patterns
        df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
        df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)
        df["quarter_sin"] = np.sin(2 * np.pi * df["quarter"] / 4)
        df["quarter_cos"] = np.cos(2 * np.pi * df["quarter"] / 4)

        # Time since start
        min_date = df[self.config.date_col].min()
        df["months_since_start"] = ((df[self.config.date_col] - min_date).dt.days / 30.44).astype(int)

        return df

    def create_lag_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create lagged features for key variables"""
        df = df.copy()
        df = df.sort_values([self.config.date_col] + self.config.id_cols).reset_index(drop=True)

        # Key variables to lag
        lag_vars = [
            "noi", "value", "cap_rate", "vacancy_rate", "rent_psf",
            "fed_rate", "inflation", "gdp_growth", "unemployment", "credit_spread",
            "total_return", "effective_rent", "opex_rate"
        ]

        # Create lags for each variable
        for var in lag_vars:
            if var in df.columns:
                for lag in self.config.lag_periods:
                    try:
                        # Group by market and asset type for proper lagging
                        lagged_values = df.groupby(self.config.id_cols, group_keys=False)[var].shift(lag)
                        df[f"{var}_lag{lag}"] = lagged_values.values  # Extract values to avoid index issues
                    except Exception as e:
                        self.logger.warning(f"Error creating lag feature {var}_lag{lag}: {str(e)}")
                        # Fallback: simple shift without grouping
                        df[f"{var}_lag{lag}"] = df[var].shift(lag)

        return df

    def create_rolling_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create rolling window statistics"""
        df = df.copy()
        df = df.sort_values([self.config.date_col] + self.config.id_cols).reset_index(drop=True)

        # Variables for rolling features
        rolling_vars = [
            "total_return", "noi", "cap_rate", "vacancy_rate", "rent_psf",
            "fed_rate", "gdp_growth", "inflation"
        ]

        for var in rolling_vars:
            if var in df.columns:
                for window in self.config.rolling_windows:
                    try:
                        # Use transform to maintain proper alignment
                        grouped = df.groupby(self.config.id_cols, group_keys=False)[var]

                        # Rolling mean and std
                        df[f"{var}_roll_mean_{window}"] = grouped.rolling(
                            window=window, min_periods=1
                        ).mean().reset_index(level=self.config.id_cols, drop=True)

                        df[f"{var}_roll_std_{window}"] = grouped.rolling(
                            window=window, min_periods=1
                        ).std().reset_index(level=self.config.id_cols, drop=True)

                        # Rolling min/max
                        df[f"{var}_roll_min_{window}"] = grouped.rolling(
                            window=window, min_periods=1
                        ).min().reset_index(level=self.config.id_cols, drop=True)

                        df[f"{var}_roll_max_{window}"] = grouped.rolling(
                            window=window, min_periods=1
                        ).max().reset_index(level=self.config.id_cols, drop=True)

                    except Exception as e:
                        self.logger.warning(f"Error creating rolling feature {var}_roll_*_{window}: {str(e)}")
                        # Fallback: simple rolling without grouping
                        try:
                            df[f"{var}_roll_mean_{window}"] = df[var].rolling(window=window, min_periods=1).mean()
                            df[f"{var}_roll_std_{window}"] = df[var].rolling(window=window, min_periods=1).std()
                            df[f"{var}_roll_min_{window}"] = df[var].rolling(window=window, min_periods=1).min()
                            df[f"{var}_roll_max_{window}"] = df[var].rolling(window=window, min_periods=1).max()
                        except Exception as e2:
                            self.logger.warning(f"Fallback rolling feature creation also failed for {var}: {str(e2)}")

        return df

    def create_interaction_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create interaction and derived features"""
        df = df.copy()

        # Economic interaction terms
        if all(col in df.columns for col in ["gdp_growth", "fed_rate"]):
            df["gdp_fed_interaction"] = df["gdp_growth"] * df["fed_rate"]

        if all(col in df.columns for col in ["unemployment", "inflation"]):
            df["unemployment_inflation_ratio"] = df["unemployment"] / np.maximum(df["inflation"], 0.1)

        # Property-specific ratios and metrics
        if all(col in df.columns for col in ["noi", "value"]):
            df["noi_yield"] = df["noi"] / np.maximum(df["value"], 1e-6)

        if all(col in df.columns for col in ["effective_rent", "sqft"]):
            df["rent_per_sqft_eff"] = df["effective_rent"] / np.maximum(df["sqft"], 1)

        if all(col in df.columns for col in ["annual_debt_service", "noi"]):
            df["debt_service_coverage"] = df["noi"] / np.maximum(df["annual_debt_service"], 1e-6)

        # Market timing features
        if "age" in df.columns:
            df["age_squared"] = df["age"] ** 2
            df["log_age"] = np.log1p(df["age"])

        # Leverage metrics
        if all(col in df.columns for col in ["loan_amt", "value"]):
            df["ltv_calculated"] = df["loan_amt"] / np.maximum(df["value"], 1e-6)

        return df

    def create_market_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create market-level aggregated features"""
        df = df.copy()

        try:
            # Market-level statistics by date
            market_agg = df.groupby(["date", "market"]).agg({
                "total_return": ["mean", "std", "count"],
                "cap_rate": ["mean", "median"],
                "vacancy_rate": ["mean"],
                "rent_psf": ["mean", "std"]
            })

            # Flatten column names
            market_agg.columns = ["_".join(col).strip() for col in market_agg.columns.values]
            market_agg = market_agg.reset_index()

            # Rename for clarity
            rename_dict = {
                "total_return_mean": "market_avg_return",
                "total_return_std": "market_return_volatility",
                "total_return_count": "market_property_count",
                "cap_rate_mean": "market_avg_cap_rate",
                "cap_rate_median": "market_median_cap_rate",
                "vacancy_rate_mean": "market_avg_vacancy",
                "rent_psf_mean": "market_avg_rent_psf",
                "rent_psf_std": "market_rent_volatility"
            }

            market_agg = market_agg.rename(columns=rename_dict)

            # Fill NaN values in volatility measures
            market_agg["market_return_volatility"] = market_agg["market_return_volatility"].fillna(0)
            market_agg["market_rent_volatility"] = market_agg["market_rent_volatility"].fillna(0)

            # Merge back to main dataframe
            df = df.merge(market_agg, on=["date", "market"], how="left")

        except Exception as e:
            self.logger.warning(f"Error creating market features: {str(e)}")
            # Skip market features if they fail
            pass

        return df

    def engineer_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Apply all feature engineering steps"""
        self.logger.info("Starting feature engineering...")
        original_cols = len(df.columns)

        # Apply feature engineering steps
        df = self.create_time_features(df)

        if self.config.enable_advanced_features:
            df = self.create_lag_features(df)
            df = self.create_rolling_features(df)
            df = self.create_interaction_features(df)
            df = self.create_market_features(df)

        # Remove rows with NaN values created by lagging
        df = df.dropna().reset_index(drop=True)

        new_cols = len(df.columns)
        self.logger.info(f"Feature engineering complete. Added {new_cols - original_cols} features. "
                        f"Dataset shape: {df.shape}")

        return df


# Advanced Model Framework

class ModelFactory:
    """Factory for creating different types of models"""

    @staticmethod
    def create_preprocessor(numeric_cols: List[str], categorical_cols: List[str]) -> ColumnTransformer:
        """Create preprocessing pipeline"""
        try:
            # Try newer scikit-learn API
            ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
        except TypeError:
            # Fallback for older versions
            ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

        preprocessor = ColumnTransformer([
            ("num", RobustScaler(), numeric_cols),  # More robust to outliers
            ("cat", ohe, categorical_cols)
        ])

        return preprocessor

    @staticmethod
    def create_model(model_type: str, random_state: int = 42) -> Any:
        """Create model based on type"""
        models = {
            'gradient_boosting': GradientBoostingRegressor(
                random_state=random_state,
                learning_rate=0.05,
                n_estimators=500,
                max_depth=4,
                subsample=0.8,
                validation_fraction=0.1,
                n_iter_no_change=50
            ),
            'random_forest': RandomForestRegressor(
                random_state=random_state,
                n_estimators=300,
                max_depth=8,
                min_samples_split=10,
                min_samples_leaf=4,
                bootstrap=True,
                oob_score=True
            ),
            'ridge': Ridge(alpha=1.0),
            'elastic_net': ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=random_state)
        }

        return models.get(model_type, models['gradient_boosting'])

    @staticmethod
    def get_hyperparameter_grid(model_type: str) -> Dict[str, List]:
        """Get hyperparameter grid for model tuning"""
        grids = {
            'gradient_boosting': {
                'model__learning_rate': [0.01, 0.05, 0.1],
                'model__max_depth': [3, 4, 6],
                'model__n_estimators': [300, 500],
                'model__subsample': [0.8, 0.9]
            },
            'random_forest': {
                'model__n_estimators': [200, 300],
                'model__max_depth': [6, 8, 10],
                'model__min_samples_split': [5, 10],
                'model__min_samples_leaf': [2, 4]
            },
            'ridge': {
                'model__alpha': [0.1, 1.0, 10.0, 100.0]
            }
        }

        return grids.get(model_type, {})


class AdvancedROIEngine:
    """Main ROI prediction engine with advanced capabilities"""

    def __init__(self, config: ROIEngineConfig, logger: logging.Logger):
        self.config = config
        self.logger = logger
        self.feature_engineer = FeatureEngineer(config, logger)
        self.models = {}
        self.results = {}

    def load_or_generate_data(self) -> pd.DataFrame:
        """Load existing data or generate new synthetic data"""
        if self.config.data_path.exists():
            self.logger.info(f"Loading existing data from {self.config.data_path}")
            df = pd.read_csv(self.config.data_path)
            df[self.config.date_col] = pd.to_datetime(df[self.config.date_col])
        else:
            self.logger.info("Generating new synthetic dataset")
            generator = SyntheticDataGenerator(self.config, self.logger)
            df = generator.generate_dataset(n_periods=120)
            df.to_csv(self.config.data_path, index=False)
            self.logger.info(f"Saved synthetic data to {self.config.data_path}")

        return df

    def prepare_data(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, List[str], List[str]]:
        """Prepare data with feature engineering"""
        # Apply feature engineering
        df_processed = self.feature_engineer.engineer_features(df)

        # Identify feature columns
        exclude_cols = [self.config.target, self.config.date_col, "value_next", "year_month"]

        numeric_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
        numeric_cols = [c for c in numeric_cols if c not in exclude_cols]

        categorical_cols = [c for c in self.config.id_cols if c in df_processed.columns]
        for c in df_processed.select_dtypes(include=["object", "category"]).columns:
            if c not in [self.config.target, self.config.date_col] and c not in categorical_cols:
                categorical_cols.append(c)

        self.logger.info(f"Prepared {len(numeric_cols)} numeric and {len(categorical_cols)} categorical features")

        return df_processed, numeric_cols, categorical_cols

    def train_single_model(self, X: pd.DataFrame, y: np.ndarray,
                          model_type: str, numeric_cols: List[str],
                          categorical_cols: List[str]) -> Dict[str, Any]:
        """Train a single model with cross-validation"""

        # Create preprocessing and model pipeline
        preprocessor = ModelFactory.create_preprocessor(numeric_cols, categorical_cols)
        model = ModelFactory.create_model(model_type, self.config.random_state)
        pipeline = Pipeline([("preprocess", preprocessor), ("model", model)])

        # Hyperparameter tuning if enabled
        if self.config.enable_hyperparameter_tuning:
            param_grid = ModelFactory.get_hyperparameter_grid(model_type)
            if param_grid:
                self.logger.info(f"Performing hyperparameter tuning for {model_type}")

                # Use fewer splits for tuning to speed up
                tscv_tune = TimeSeriesSplit(n_splits=3)
                grid_search = GridSearchCV(
                    pipeline, param_grid, cv=tscv_tune,
                    scoring='neg_mean_squared_error',
                    n_jobs=self.config.n_jobs, verbose=1
                )
                grid_search.fit(X, y)
                pipeline = grid_search.best_estimator_
                self.logger.info(f"Best parameters for {model_type}: {grid_search.best_params_}")

        # Cross-validation evaluation
        tscv = TimeSeriesSplit(n_splits=self.config.n_splits)
        cv_scores = {'mae': [], 'rmse': [], 'r2': [], 'mape': []}
        y_true_all, y_pred_all = [], []

        for fold_idx, (train_idx, test_idx) in enumerate(tscv.split(X)):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # Fit pipeline
            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)

            # Calculate metrics
            mae = mean_absolute_error(y_test, y_pred)
            rmse = mean_squared_error(y_test, y_pred, squared=False)
            r2 = r2_score(y_test, y_pred)
            mape = np.mean(np.abs((y_test - y_pred) / np.clip(np.abs(y_test), 1e-6, None))) * 100

            cv_scores['mae'].append(mae)
            cv_scores['rmse'].append(rmse)
            cv_scores['r2'].append(r2)
            cv_scores['mape'].append(mape)

            y_true_all.extend(y_test.tolist())
            y_pred_all.extend(y_pred.tolist())

            self.logger.info(f"{model_type} Fold {fold_idx}: MAE={mae:.4f}, RMSE={rmse:.4f}, R²={r2:.4f}")

        # Fit final model on all data
        pipeline.fit(X, y)

        # Calculate average metrics
        avg_metrics = {metric: np.mean(scores) for metric, scores in cv_scores.items()}
        std_metrics = {metric: np.std(scores) for metric, scores in cv_scores.items()}

        return {
            'pipeline': pipeline,
            'cv_metrics': cv_scores,
            'avg_metrics': avg_metrics,
            'std_metrics': std_metrics,
            'y_true': y_true_all,
            'y_pred': y_pred_all,
            'model_type': model_type
        }

    def train_models(self, df: pd.DataFrame) -> Dict[str, Dict[str, Any]]:
        """Train multiple models and compare performance"""
        self.logger.info("Starting model training...")

        # Prepare data
        df_processed, numeric_cols, categorical_cols = self.prepare_data(df)

        # Sort by date for proper time series splitting
        df_processed = df_processed.sort_values(self.config.date_col).reset_index(drop=True)

        X = df_processed.drop(columns=[self.config.target])
        y = df_processed[self.config.target].values

        self.logger.info(f"Training dataset shape: X={X.shape}, y={y.shape}")

        # Train each model
        results = {}
        for model_type in self.config.models_to_try:
            self.logger.info(f"Training {model_type} model...")

            try:
                model_results = self.train_single_model(
                    X, y, model_type, numeric_cols, categorical_cols
                )
                results[model_type] = model_results

                avg_mae = model_results['avg_metrics']['mae']
                avg_r2 = model_results['avg_metrics']['r2']
                self.logger.info(f"{model_type} completed - Average MAE: {avg_mae:.4f}, R²: {avg_r2:.4f}")

            except Exception as e:
                self.logger.error(f"Error training {model_type}: {str(e)}")
                continue

        # Select best model based on R²
        if results:
            best_model = max(results.keys(), key=lambda k: results[k]['avg_metrics']['r2'])
            self.logger.info(f"Best model: {best_model} (R² = {results[best_model]['avg_metrics']['r2']:.4f})")

            # Store best model
            self.best_model_name = best_model
            self.best_model = results[best_model]['pipeline']

        self.results = results
        return results

    def create_visualizations(self):
        """Create comprehensive visualization suite"""
        if not self.results:
            self.logger.warning("No results available for visualization")
            return

        self.logger.info("Creating visualizations...")

        # Set up plotting style
        plt.style.use('seaborn-v0_8')
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

        # 1. Model Comparison
        self._plot_model_comparison()

        # 2. Feature Importance (for best model)
        if hasattr(self, 'best_model'):
            self._plot_feature_importance()

        # 3. Residual Analysis
        self._plot_residual_analysis()

        # 4. Prediction vs Actual
        self._plot_predictions()

        # 5. Time Series Analysis
        self._plot_time_series_performance()

        self.logger.info("Visualizations completed")

    def _plot_model_comparison(self):
        """Plot model performance comparison"""
        if len(self.results) < 2:
            return

        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')

        models = list(self.results.keys())
        metrics = ['mae', 'rmse', 'r2', 'mape']
        metric_names = ['Mean Absolute Error', 'Root Mean Squared Error', 'R² Score', 'Mean Absolute Percentage Error']

        for idx, (metric, name) in enumerate(zip(metrics, metric_names)):
            ax = axes[idx//2, idx%2]

            means = [self.results[model]['avg_metrics'][metric] for model in models]
            stds = [self.results[model]['std_metrics'][metric] for model in models]

            bars = ax.bar(models, means, yerr=stds, capsize=5, alpha=0.8)
            ax.set_title(name, fontweight='bold')
            ax.set_ylabel(name)

            # Rotate x-axis labels if needed
            if len(max(models, key=len)) > 8:
                ax.tick_params(axis='x', rotation=45)

            # Add value labels on bars
            for bar, mean in zip(bars, means):
                height = bar.get_height()
                ax.annotate(f'{mean:.3f}', xy=(bar.get_x() + bar.get_width()/2, height),
                           xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

        plt.tight_layout()
        plt.savefig(self.config.plots_dir / f'model_comparison.{self.config.plot_format}',
                   dpi=self.config.plot_dpi, bbox_inches='tight')
        plt.close()

    def _plot_feature_importance(self):
        """Plot feature importance for the best model"""
        if not hasattr(self, 'best_model'):
            return

        # Extract model from pipeline
        model = self.best_model.named_steps['model']
        preprocessor = self.best_model.named_steps['preprocess']

        if not hasattr(model, 'feature_importances_'):
            return

        # Get feature names
        numeric_features = preprocessor.transformers_[0][2]
        cat_encoder = preprocessor.transformers_[1][1]
        cat_features = preprocessor.transformers_[1][2]

        if hasattr(cat_encoder, 'get_feature_names_out'):
            cat_feature_names = cat_encoder.get_feature_names_out(cat_features).tolist()
        else:
            cat_feature_names = []

        feature_names = list(numeric_features) + cat_feature_names
        importances = model.feature_importances_

        # Sort by importance
        indices = np.argsort(importances)[::-1]
        top_n = min(20, len(indices))

        plt.figure(figsize=(12, 8))
        plt.bar(range(top_n), importances[indices[:top_n]], alpha=0.8)
        plt.title(f'Top {top_n} Feature Importances - {self.best_model_name.title()}',
                 fontsize=14, fontweight='bold')
        plt.xlabel('Features')
        plt.ylabel('Importance')

        feature_labels = [feature_names[i] if i < len(feature_names) else f'Feature_{i}'
                         for i in indices[:top_n]]
        plt.xticks(range(top_n), feature_labels, rotation=90)

        plt.tight_layout()
        plt.savefig(self.config.plots_dir / f'feature_importance.{self.config.plot_format}',
                   dpi=self.config.plot_dpi, bbox_inches='tight')
        plt.close()

    def _plot_residual_analysis(self):
        """Create residual analysis plots"""
        if not hasattr(self, 'best_model_name'):
            return

        best_results = self.results[self.best_model_name]
        y_true = np.array(best_results['y_true'])
        y_pred = np.array(best_results['y_pred'])
        residuals = y_true - y_pred

        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle(f'Residual Analysis - {self.best_model_name.title()}',
                    fontsize=16, fontweight='bold')

        # 1. Residuals vs Predicted
        axes[0,0].scatter(y_pred, residuals, alpha=0.6, s=20)
        axes[0,0].axhline(y=0, color='red', linestyle='--')
        axes[0,0].set_xlabel('Predicted Values')
        axes[0,0].set_ylabel('Residuals')
        axes[0,0].set_title('Residuals vs Predicted')

        # 2. Q-Q Plot
        from scipy import stats
        stats.probplot(residuals, dist="norm", plot=axes[0,1])
        axes[0,1].set_title('Q-Q Plot (Normal Distribution)')

        # 3. Histogram of Residuals
        axes[1,0].hist(residuals, bins=30, alpha=0.7, density=True)
        axes[1,0].set_xlabel('Residuals')
        axes[1,0].set_ylabel('Density')
        axes[1,0].set_title('Distribution of Residuals')

        # Add normal curve overlay
        mu, sigma = stats.norm.fit(residuals)
        x = np.linspace(residuals.min(), residuals.max(), 100)
        axes[1,0].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', lw=2, label='Normal Fit')
        axes[1,0].legend()

        # 4. Scale-Location Plot
        sqrt_abs_resid = np.sqrt(np.abs(residuals))
        axes[1,1].scatter(y_pred, sqrt_abs_resid, alpha=0.6, s=20)
        axes[1,1].set_xlabel('Predicted Values')
        axes[1,1].set_ylabel('√|Residuals|')
        axes[1,1].set_title('Scale-Location Plot')

        plt.tight_layout()
        plt.savefig(self.config.plots_dir / f'residual_analysis.{self.config.plot_format}',
                   dpi=self.config.plot_dpi, bbox_inches='tight')
        plt.close()

    def _plot_predictions(self):
        """Plot predictions vs actual values"""
        if not hasattr(self, 'best_model_name'):
            return

        best_results = self.results[self.best_model_name]
        y_true = np.array(best_results['y_true'])
        y_pred = np.array(best_results['y_pred'])

        plt.figure(figsize=(10, 8))

        # Scatter plot
        plt.scatter(y_true, y_pred, alpha=0.6, s=20)

        # Perfect prediction line
        min_val, max_val = min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')

        # Calculate R²
        r2 = r2_score(y_true, y_pred)
        plt.text(0.05, 0.95, f'R² = {r2:.4f}', transform=plt.gca().transAxes,
                fontsize=12, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.title(f'Predictions vs Actual - {self.best_model_name.title()}',
                 fontsize=14, fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(self.config.plots_dir / f'predictions_vs_actual.{self.config.plot_format}',
                   dpi=self.config.plot_dpi, bbox_inches='tight')
        plt.close()

    def _plot_time_series_performance(self):
        """Plot model performance over time"""
        # This would require additional date tracking in cross-validation
        # For now, we'll create a placeholder
        pass

    def save_results(self):
        """Save all results and models"""
        self.logger.info("Saving results...")

        # Save model comparison metrics
        comparison_data = []
        for model_name, results in self.results.items():
            row = {'model': model_name}
            row.update(results['avg_metrics'])
            row.update({f'{k}_std': v for k, v in results['std_metrics'].items()})
            comparison_data.append(row)

        comparison_df = pd.DataFrame(comparison_data)
        comparison_df.to_csv(self.config.artifacts_dir / 'model_comparison.csv', index=False)

        # Save detailed results
        with open(self.config.artifacts_dir / 'detailed_results.json', 'w') as f:
            # Convert results to JSON-serializable format
            json_results = {}
            for model_name, results in self.results.items():
                json_results[model_name] = {
                    'cv_metrics': results['cv_metrics'],
                    'avg_metrics': results['avg_metrics'],
                    'std_metrics': results['std_metrics'],
                    'model_type': results['model_type']
                    # Note: pipeline and predictions are not JSON serializable
                }
            json.dump(json_results, f, indent=2)

        # Save best model
        if hasattr(self, 'best_model'):
            joblib.dump(self.best_model, self.config.artifacts_dir / 'best_model.joblib')

            # Save predictions from best model
            best_results = self.results[self.best_model_name]
            predictions_df = pd.DataFrame({
                'y_true': best_results['y_true'],
                'y_pred': best_results['y_pred'],
                'residuals': np.array(best_results['y_true']) - np.array(best_results['y_pred'])
            })
            predictions_df.to_csv(self.config.artifacts_dir / 'best_model_predictions.csv', index=False)

        self.logger.info(f"Results saved to {self.config.artifacts_dir}")

    def run_complete_analysis(self) -> Dict[str, Any]:
        """Run the complete ROI analysis pipeline"""
        self.logger.info("Starting complete ROI analysis...")

        try:
            # Load or generate data
            df = self.load_or_generate_data()

            # Train models
            results = self.train_models(df)

            # Create visualizations
            if self.config.save_plots:
                self.create_visualizations()

            # Save results
            self.save_results()

            # Print summary
            self._print_summary()

            return results

        except Exception as e:
            self.logger.error(f"Error in analysis pipeline: {str(e)}")
            raise

    def _print_summary(self):
        """Print analysis summary"""
        print("\n" + "="*60)
        print("ROI ENGINE ANALYSIS SUMMARY")
        print("="*60)

        if not self.results:
            print("No results available.")
            return

        # Model comparison
        print("\nModel Performance Comparison:")
        print("-" * 40)
        for model_name, results in self.results.items():
            metrics = results['avg_metrics']
            print(f"{model_name.upper():<20} | R²: {metrics['r2']:.4f} | MAE: {metrics['mae']:.4f} | RMSE: {metrics['rmse']:.4f}")

        # Best model details
        if hasattr(self, 'best_model_name'):
            print(f"\nBest Model: {self.best_model_name.upper()}")
            best_metrics = self.results[self.best_model_name]['avg_metrics']
            print(f"  R² Score: {best_metrics['r2']:.4f}")
            print(f"  Mean Absolute Error: {best_metrics['mae']:.4f}")
            print(f"  Root Mean Squared Error: {best_metrics['rmse']:.4f}")
            print(f"  Mean Absolute Percentage Error: {best_metrics['mape']:.2f}%")

        print(f"\nResults saved to: {self.config.artifacts_dir}")
        print("="*60)


# =========================
# 5) Main Execution
# =========================

def main():
    """Main execution function"""

    # Initialize configuration
    config = ROIEngineConfig(
        enable_advanced_features=True,
        enable_hyperparameter_tuning=True,
        models_to_try=['gradient_boosting', 'random_forest', 'ridge'],
        save_plots=True
    )

    # Setup logging
    logger = setup_logging(config)
    logger.info("Starting Advanced ROI Engine")

    try:
        # Initialize and run engine
        engine = AdvancedROIEngine(config, logger)
        results = engine.run_complete_analysis()

        logger.info("Analysis completed successfully")
        return results

    except Exception as e:
        logger.error(f"Analysis failed: {str(e)}")
        raise


# Execution

if __name__ == "__main__":
    results = main()

Fitting 3 folds for each of 36 candidates, totalling 108 fits


ERROR:ROI_Engine:Error training gradient_boosting: got an unexpected keyword argument 'squared'


Fitting 3 folds for each of 24 candidates, totalling 72 fits


ERROR:ROI_Engine:Error training random_forest: got an unexpected keyword argument 'squared'


Fitting 3 folds for each of 4 candidates, totalling 12 fits


ERROR:ROI_Engine:Error training ridge: got an unexpected keyword argument 'squared'



ROI ENGINE ANALYSIS SUMMARY
No results available.


In [7]:
# =========================
# Advanced ROI Engine for Real Estate Investments (CRE)
# Enhanced version with robust error handling, feature engineering, and model optimization
# =========================

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from datetime import datetime
from typing import List, Tuple, Dict, Any, Optional, Union
from dataclasses import dataclass, field
import logging

# ML imports
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.feature_selection import SelectFromModel
import joblib

# Suppress warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

# =========================
# 0) Configuration & Setup
# =========================

@dataclass
class ROIEngineConfig:
    """Advanced configuration for ROI Engine"""
    # Data paths
    base_dir: str = "./roi_engine_project"
    data_filename: str = "synthetic_cre_dataset.csv"

    # Model settings
    target: str = "total_return"
    date_col: str = "date"
    id_cols: List[str] = field(default_factory=lambda: ["market", "asset_type"])

    # Cross-validation
    n_splits: int = 5
    test_size_ratio: float = 0.2

    # Model parameters
    random_state: int = 42
    n_jobs: int = -1

    # Feature engineering
    enable_advanced_features: bool = True
    rolling_windows: List[int] = field(default_factory=lambda: [3, 6, 12])
    lag_periods: List[int] = field(default_factory=lambda: [1, 2, 3, 6])

    # Model selection
    models_to_try: List[str] = field(default_factory=lambda: ['gradient_boosting', 'random_forest', 'ridge'])
    enable_hyperparameter_tuning: bool = True

    # Output settings
    save_plots: bool = True
    plot_format: str = "png"
    plot_dpi: int = 300

    def __post_init__(self):
        self.data_dir = Path(self.base_dir) / "data"
        self.artifacts_dir = Path(self.base_dir) / "artifacts"
        self.plots_dir = self.artifacts_dir / "plots"

        # Create directories
        for dir_path in [self.data_dir, self.artifacts_dir, self.plots_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)

        self.data_path = self.data_dir / self.data_filename


# =========================
# 1) Logging Setup
# =========================

def setup_logging(config: ROIEngineConfig) -> logging.Logger:
    """Setup logging configuration"""
    log_file = config.artifacts_dir / "roi_engine.log"

    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )

    logger = logging.getLogger('ROI_Engine')
    return logger


# =========================
# 2) Enhanced Data Generation
# =========================

class SyntheticDataGenerator:
    """Enhanced synthetic CRE data generator with realistic market dynamics"""

    def __init__(self, config: ROIEngineConfig, logger: logging.Logger):
        self.config = config
        self.logger = logger

    def generate_macroeconomic_data(self, n_periods: int, start_date: str = "2016-01-01") -> pd.DataFrame:
        """Generate realistic macroeconomic time series"""
        np.random.seed(self.config.random_state)

        dates = pd.date_range(start_date, periods=n_periods, freq="M")
        time_trend = np.linspace(0, 8, n_periods)

        # More realistic macro indicators with autocorrelation
        inflation_base = 2.0 + 0.8 * np.sin(time_trend) + np.random.normal(0, 0.3, n_periods)
        inflation = np.cumsum(np.random.normal(0, 0.1, n_periods)) + inflation_base
        inflation = np.clip(inflation, 0.5, 6.0)

        fed_rate_changes = np.random.normal(0, 0.05, n_periods)
        fed_rate_changes[::24] += np.random.normal(0, 0.3, len(fed_rate_changes[::24]))  # Policy changes
        fed_rate = np.clip(np.cumsum(fed_rate_changes) + 1.5, 0.1, 8.0)

        gdp_base = 2.5 + 0.5 * np.cos(time_trend * 1.2)
        gdp_shocks = np.random.choice([0, 0, 0, 0, -2, 1.5], n_periods, p=[0.7, 0.1, 0.1, 0.05, 0.03, 0.02])
        gdp_growth = gdp_base + gdp_shocks + np.random.normal(0, 0.2, n_periods)
        gdp_growth = np.clip(gdp_growth, -3.0, 6.0)

        # Employment and credit conditions
        unemployment = np.clip(6.0 - 0.3 * gdp_growth + np.random.normal(0, 0.3, n_periods), 3.0, 12.0)
        credit_spread = np.clip(1.5 + 0.2 * (fed_rate - 2.0) - 0.1 * gdp_growth +
                               np.random.normal(0, 0.2, n_periods), 0.5, 4.0)

        return pd.DataFrame({
            'date': dates,
            'inflation': inflation,
            'fed_rate': fed_rate,
            'gdp_growth': gdp_growth,
            'unemployment': unemployment,
            'credit_spread': credit_spread
        })

    def generate_market_data(self, macro_df: pd.DataFrame) -> pd.DataFrame:
        """Generate market-specific real estate data"""
        n_periods = len(macro_df)

        # Market definitions with more granular characteristics
        market_params = {
            "NYC": {"base_rent": 85, "volatility": 0.15, "liquidity": 0.9, "barrier_to_entry": 0.8},
            "LA": {"base_rent": 55, "volatility": 0.18, "liquidity": 0.7, "barrier_to_entry": 0.6},
            "SF": {"base_rent": 75, "volatility": 0.22, "liquidity": 0.6, "barrier_to_entry": 0.9},
            "CHI": {"base_rent": 40, "volatility": 0.12, "liquidity": 0.8, "barrier_to_entry": 0.4},
            "DAL": {"base_rent": 35, "volatility": 0.16, "liquidity": 0.7, "barrier_to_entry": 0.3},
            "BOS": {"base_rent": 65, "volatility": 0.14, "liquidity": 0.6, "barrier_to_entry": 0.7},
            "SEA": {"base_rent": 60, "volatility": 0.20, "liquidity": 0.5, "barrier_to_entry": 0.6}
        }

        asset_type_params = {
            "Multifamily": {"alpha": 1.15, "beta_gdp": 0.3, "beta_unemployment": -0.4, "cyclicality": 0.7},
            "Office": {"alpha": 1.0, "beta_gdp": 0.5, "beta_unemployment": -0.6, "cyclicality": 1.2},
            "Industrial": {"alpha": 1.05, "beta_gdp": 0.4, "beta_unemployment": -0.2, "cyclicality": 0.8},
            "Retail": {"alpha": 0.95, "beta_gdp": 0.6, "beta_unemployment": -0.8, "cyclicality": 1.5},
            "Hotel": {"alpha": 1.3, "beta_gdp": 0.8, "beta_unemployment": -1.0, "cyclicality": 2.0}
        }

        records = []

        for _, row in macro_df.iterrows():
            # Generate multiple properties per period for more data
            n_properties = np.random.randint(8, 15)

            for _ in range(n_properties):
                market = np.random.choice(list(market_params.keys()))
                asset_type = np.random.choice(list(asset_type_params.keys()))

                mp = market_params[market]
                atp = asset_type_params[asset_type]

                # Property characteristics
                sqft = np.random.randint(25_000, 500_000)
                age = np.random.randint(1, 50)
                quality_score = np.random.uniform(0.6, 1.0)

                # Dynamic rent calculation
                base_rent = mp["base_rent"] * atp["alpha"] * quality_score

                # Economic sensitivity
                gdp_effect = atp["beta_gdp"] * (row["gdp_growth"] - 2.5) / 2.5
                unemployment_effect = atp["beta_unemployment"] * (row["unemployment"] - 6.0) / 6.0

                market_volatility = mp["volatility"] * atp["cyclicality"]
                rent_shock = np.random.normal(0, market_volatility)

                rent_psf = base_rent * (1 + gdp_effect + unemployment_effect + rent_shock)
                rent_psf = np.clip(rent_psf, base_rent * 0.5, base_rent * 2.0)

                annual_rent = rent_psf * sqft

                # Vacancy dynamics
                base_vacancy = 5.0 + np.random.uniform(-1, 3)
                vacancy_sensitivity = -unemployment_effect * 2 + (row["fed_rate"] - 2.0) * 0.5
                vacancy_rate = np.clip(base_vacancy + vacancy_sensitivity +
                                     np.random.normal(0, 1.5), 1.0, 25.0)

                effective_rent = annual_rent * (1 - vacancy_rate / 100.0)

                # Operating expenses with inflation sensitivity
                opex_base = 0.25 + 0.05 * np.random.rand()
                opex_inflation = 0.5 * (row["inflation"] - 2.0) / 2.0
                opex_rate = np.clip(opex_base + opex_inflation, 0.15, 0.45)
                opex = effective_rent * opex_rate

                noi = effective_rent - opex

                # Cap rates with risk premiums
                base_cap = 4.5 + 0.5 * (1 - mp["liquidity"]) + 0.3 * atp["cyclicality"]
                risk_premium = 0.3 * (row["fed_rate"] - 2.0) + 0.2 * row["credit_spread"]
                market_risk = -0.1 * (row["gdp_growth"] - 2.5) / 2.5

                cap_rate = np.clip(base_cap + risk_premium + market_risk +
                                  np.random.normal(0, 0.2), 3.0, 12.0)

                value = noi / (cap_rate / 100.0) if cap_rate > 0 else 0

                # Financing
                ltv_base = 0.65
                ltv_adjustment = -0.05 * (row["fed_rate"] - 2.0) / 2.0 - 0.03 * row["credit_spread"]
                ltv = np.clip(ltv_base + ltv_adjustment + np.random.normal(0, 0.05), 0.4, 0.8)

                loan_amt = value * ltv
                coupon = (row["fed_rate"] + row["credit_spread"] + 1.5 +
                         0.5 * atp["cyclicality"] + np.random.normal(0, 0.3)) / 100.0
                coupon = np.clip(coupon, 0.02, 0.12)

                annual_debt_service = loan_amt * coupon

                # Returns calculation with market dynamics
                market_appreciation = (0.02 + 0.5 * (row["gdp_growth"] - 2.5) / 2.5 -
                                     0.3 * (row["fed_rate"] - 2.0) / 2.0)
                asset_specific = atp["beta_gdp"] * (row["gdp_growth"] - 2.5) / 10.0
                idiosyncratic = np.random.normal(0, market_volatility * 0.5)

                appreciation_rate = market_appreciation + asset_specific + idiosyncratic
                value_next = value * (1 + appreciation_rate)

                equity = np.maximum(value - loan_amt, 1e-6)
                cash_flow = noi - annual_debt_service
                total_return = (cash_flow + (value_next - value)) / equity

                # Additional metrics
                dscr = noi / annual_debt_service if annual_debt_service > 0 else np.inf
                cash_on_cash = cash_flow / equity

                records.append({
                    "date": row["date"],
                    "market": market,
                    "asset_type": asset_type,
                    "sqft": sqft,
                    "age": age,
                    "quality_score": quality_score,
                    "rent_psf": rent_psf,
                    "annual_rent": annual_rent,
                    "vacancy_rate": vacancy_rate,
                    "effective_rent": effective_rent,
                    "opex_rate": opex_rate,
                    "opex": opex,
                    "noi": noi,
                    "cap_rate": cap_rate,
                    "value": value,
                    "ltv": ltv,
                    "loan_amt": loan_amt,
                    "coupon": coupon * 100.0,
                    "annual_debt_service": annual_debt_service,
                    "equity": equity,
                    "value_next": value_next,
                    "total_return": total_return,
                    "dscr": dscr,
                    "cash_on_cash": cash_on_cash,
                    **{k: row[k] for k in ["inflation", "fed_rate", "gdp_growth", "unemployment", "credit_spread"]}
                })

        return pd.DataFrame(records)

    def generate_dataset(self, n_periods: int = 120) -> pd.DataFrame:
        """Generate complete synthetic dataset"""
        self.logger.info(f"Generating synthetic dataset with {n_periods} periods")

        # Generate macroeconomic data
        macro_df = self.generate_macroeconomic_data(n_periods)

        # Generate market data
        market_df = self.generate_market_data(macro_df)

        self.logger.info(f"Generated {len(market_df)} property records")
        return market_df


# =========================
# 3) Advanced Feature Engineering
# =========================

class FeatureEngineer:
    """Advanced feature engineering for real estate data"""

    def __init__(self, config: ROIEngineConfig, logger: logging.Logger):
        self.config = config
        self.logger = logger

    def create_time_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create comprehensive time-based features"""
        df = df.copy()

        # Basic time features
        df["year"] = df[self.config.date_col].dt.year
        df["month"] = df[self.config.date_col].dt.month
        df["quarter"] = df[self.config.date_col].dt.quarter
        df["year_month"] = df[self.config.date_col].dt.to_period('M').astype(str)

        # Cyclical encoding for seasonal patterns
        df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
        df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)
        df["quarter_sin"] = np.sin(2 * np.pi * df["quarter"] / 4)
        df["quarter_cos"] = np.cos(2 * np.pi * df["quarter"] / 4)

        # Time since start
        min_date = df[self.config.date_col].min()
        df["months_since_start"] = ((df[self.config.date_col] - min_date).dt.days / 30.44).astype(int)

        return df

    def create_lag_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create lagged features for key variables"""
        df = df.copy()
        df = df.sort_values([self.config.date_col] + self.config.id_cols).reset_index(drop=True)

        # Key variables to lag
        lag_vars = [
            "noi", "value", "cap_rate", "vacancy_rate", "rent_psf",
            "fed_rate", "inflation", "gdp_growth", "unemployment", "credit_spread",
            "total_return", "effective_rent", "opex_rate"
        ]

        # Create lags for each variable
        for var in lag_vars:
            if var in df.columns:
                for lag in self.config.lag_periods:
                    try:
                        # Group by market and asset type for proper lagging
                        lagged_values = df.groupby(self.config.id_cols, group_keys=False)[var].shift(lag)
                        df[f"{var}_lag{lag}"] = lagged_values.values  # Extract values to avoid index issues
                    except Exception as e:
                        self.logger.warning(f"Error creating lag feature {var}_lag{lag}: {str(e)}")
                        # Fallback: simple shift without grouping
                        df[f"{var}_lag{lag}"] = df[var].shift(lag)

        return df

    def create_rolling_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create rolling window statistics"""
        df = df.copy()
        df = df.sort_values([self.config.date_col] + self.config.id_cols).reset_index(drop=True)

        # Variables for rolling features
        rolling_vars = [
            "total_return", "noi", "cap_rate", "vacancy_rate", "rent_psf",
            "fed_rate", "gdp_growth", "inflation"
        ]

        for var in rolling_vars:
            if var in df.columns:
                for window in self.config.rolling_windows:
                    try:
                        # Use transform to maintain proper alignment
                        grouped = df.groupby(self.config.id_cols, group_keys=False)[var]

                        # Rolling mean and std
                        df[f"{var}_roll_mean_{window}"] = grouped.rolling(
                            window=window, min_periods=1
                        ).mean().reset_index(level=self.config.id_cols, drop=True)

                        df[f"{var}_roll_std_{window}"] = grouped.rolling(
                            window=window, min_periods=1
                        ).std().reset_index(level=self.config.id_cols, drop=True)

                        # Rolling min/max
                        df[f"{var}_roll_min_{window}"] = grouped.rolling(
                            window=window, min_periods=1
                        ).min().reset_index(level=self.config.id_cols, drop=True)

                        df[f"{var}_roll_max_{window}"] = grouped.rolling(
                            window=window, min_periods=1
                        ).max().reset_index(level=self.config.id_cols, drop=True)

                    except Exception as e:
                        self.logger.warning(f"Error creating rolling feature {var}_roll_*_{window}: {str(e)}")
                        # Fallback: simple rolling without grouping
                        try:
                            df[f"{var}_roll_mean_{window}"] = df[var].rolling(window=window, min_periods=1).mean()
                            df[f"{var}_roll_std_{window}"] = df[var].rolling(window=window, min_periods=1).std()
                            df[f"{var}_roll_min_{window}"] = df[var].rolling(window=window, min_periods=1).min()
                            df[f"{var}_roll_max_{window}"] = df[var].rolling(window=window, min_periods=1).max()
                        except Exception as e2:
                            self.logger.warning(f"Fallback rolling feature creation also failed for {var}: {str(e2)}")

        return df

    def create_interaction_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create interaction and derived features"""
        df = df.copy()

        # Economic interaction terms
        if all(col in df.columns for col in ["gdp_growth", "fed_rate"]):
            df["gdp_fed_interaction"] = df["gdp_growth"] * df["fed_rate"]

        if all(col in df.columns for col in ["unemployment", "inflation"]):
            df["unemployment_inflation_ratio"] = df["unemployment"] / np.maximum(df["inflation"], 0.1)

        # Property-specific ratios and metrics
        if all(col in df.columns for col in ["noi", "value"]):
            df["noi_yield"] = df["noi"] / np.maximum(df["value"], 1e-6)

        if all(col in df.columns for col in ["effective_rent", "sqft"]):
            df["rent_per_sqft_eff"] = df["effective_rent"] / np.maximum(df["sqft"], 1)

        if all(col in df.columns for col in ["annual_debt_service", "noi"]):
            df["debt_service_coverage"] = df["noi"] / np.maximum(df["annual_debt_service"], 1e-6)

        # Market timing features
        if "age" in df.columns:
            df["age_squared"] = df["age"] ** 2
            df["log_age"] = np.log1p(df["age"])

        # Leverage metrics
        if all(col in df.columns for col in ["loan_amt", "value"]):
            df["ltv_calculated"] = df["loan_amt"] / np.maximum(df["value"], 1e-6)

        return df

    def create_market_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create market-level aggregated features"""
        df = df.copy()

        try:
            # Market-level statistics by date
            market_agg = df.groupby(["date", "market"]).agg({
                "total_return": ["mean", "std", "count"],
                "cap_rate": ["mean", "median"],
                "vacancy_rate": ["mean"],
                "rent_psf": ["mean", "std"]
            })

            # Flatten column names
            market_agg.columns = ["_".join(col).strip() for col in market_agg.columns.values]
            market_agg = market_agg.reset_index()

            # Rename for clarity
            rename_dict = {
                "total_return_mean": "market_avg_return",
                "total_return_std": "market_return_volatility",
                "total_return_count": "market_property_count",
                "cap_rate_mean": "market_avg_cap_rate",
                "cap_rate_median": "market_median_cap_rate",
                "vacancy_rate_mean": "market_avg_vacancy",
                "rent_psf_mean": "market_avg_rent_psf",
                "rent_psf_std": "market_rent_volatility"
            }

            market_agg = market_agg.rename(columns=rename_dict)

            # Fill NaN values in volatility measures
            market_agg["market_return_volatility"] = market_agg["market_return_volatility"].fillna(0)
            market_agg["market_rent_volatility"] = market_agg["market_rent_volatility"].fillna(0)

            # Merge back to main dataframe
            df = df.merge(market_agg, on=["date", "market"], how="left")

        except Exception as e:
            self.logger.warning(f"Error creating market features: {str(e)}")
            # Skip market features if they fail
            pass

        return df

    def engineer_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Apply all feature engineering steps"""
        self.logger.info("Starting feature engineering...")
        original_cols = len(df.columns)

        # Apply feature engineering steps
        df = self.create_time_features(df)

        if self.config.enable_advanced_features:
            df = self.create_lag_features(df)
            df = self.create_rolling_features(df)
            df = self.create_interaction_features(df)
            df = self.create_market_features(df)

        # Remove rows with NaN values created by lagging
        df = df.dropna().reset_index(drop=True)

        new_cols = len(df.columns)
        self.logger.info(f"Feature engineering complete. Added {new_cols - original_cols} features. "
                        f"Dataset shape: {df.shape}")

        return df


# =========================
# 4) Advanced Model Framework
# =========================

class ModelFactory:
    """Factory for creating different types of models"""

    @staticmethod
    def create_preprocessor(numeric_cols: List[str], categorical_cols: List[str]) -> ColumnTransformer:
        """Create preprocessing pipeline"""
        try:
            # Try newer scikit-learn API
            ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
        except TypeError:
            # Fallback for older versions
            ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

        preprocessor = ColumnTransformer([
            ("num", RobustScaler(), numeric_cols),  # More robust to outliers
            ("cat", ohe, categorical_cols)
        ])

        return preprocessor

    @staticmethod
    def create_model(model_type: str, random_state: int = 42) -> Any:
        """Create model based on type"""
        models = {
            'gradient_boosting': GradientBoostingRegressor(
                random_state=random_state,
                learning_rate=0.05,
                n_estimators=500,
                max_depth=4,
                subsample=0.8,
                validation_fraction=0.1,
                n_iter_no_change=50
            ),
            'random_forest': RandomForestRegressor(
                random_state=random_state,
                n_estimators=300,
                max_depth=8,
                min_samples_split=10,
                min_samples_leaf=4,
                bootstrap=True,
                oob_score=True
            ),
            'ridge': Ridge(alpha=1.0),
            'elastic_net': ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=random_state)
        }

        return models.get(model_type, models['gradient_boosting'])

    @staticmethod
    def get_hyperparameter_grid(model_type: str) -> Dict[str, List]:
        """Get hyperparameter grid for model tuning"""
        grids = {
            'gradient_boosting': {
                'model__learning_rate': [0.01, 0.05, 0.1],
                'model__max_depth': [3, 4, 6],
                'model__n_estimators': [300, 500],
                'model__subsample': [0.8, 0.9]
            },
            'random_forest': {
                'model__n_estimators': [200, 300],
                'model__max_depth': [6, 8, 10],
                'model__min_samples_split': [5, 10],
                'model__min_samples_leaf': [2, 4]
            },
            'ridge': {
                'model__alpha': [0.1, 1.0, 10.0, 100.0]
            }
        }

        return grids.get(model_type, {})


class AdvancedROIEngine:
    """Main ROI prediction engine with advanced capabilities"""

    def __init__(self, config: ROIEngineConfig, logger: logging.Logger):
        self.config = config
        self.logger = logger
        self.feature_engineer = FeatureEngineer(config, logger)
        self.models = {}
        self.results = {}

    def load_or_generate_data(self) -> pd.DataFrame:
        """Load existing data or generate new synthetic data"""
        if self.config.data_path.exists():
            self.logger.info(f"Loading existing data from {self.config.data_path}")
            df = pd.read_csv(self.config.data_path)
            df[self.config.date_col] = pd.to_datetime(df[self.config.date_col])
        else:
            self.logger.info("Generating new synthetic dataset")
            generator = SyntheticDataGenerator(self.config, self.logger)
            df = generator.generate_dataset(n_periods=120)
            df.to_csv(self.config.data_path, index=False)
            self.logger.info(f"Saved synthetic data to {self.config.data_path}")

        return df

    def prepare_data(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, List[str], List[str]]:
        """Prepare data with feature engineering"""
        # Apply feature engineering
        df_processed = self.feature_engineer.engineer_features(df)

        # Identify feature columns
        exclude_cols = [self.config.target, self.config.date_col, "value_next", "year_month"]

        numeric_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
        numeric_cols = [c for c in numeric_cols if c not in exclude_cols]

        categorical_cols = [c for c in self.config.id_cols if c in df_processed.columns]
        for c in df_processed.select_dtypes(include=["object", "category"]).columns:
            if c not in [self.config.target, self.config.date_col] and c not in categorical_cols:
                categorical_cols.append(c)

        self.logger.info(f"Prepared {len(numeric_cols)} numeric and {len(categorical_cols)} categorical features")

        return df_processed, numeric_cols, categorical_cols

    def train_single_model(self, X: pd.DataFrame, y: np.ndarray,
                          model_type: str, numeric_cols: List[str],
                          categorical_cols: List[str]) -> Dict[str, Any]:
        """Train a single model with cross-validation"""

        # Create preprocessing and model pipeline
        preprocessor = ModelFactory.create_preprocessor(numeric_cols, categorical_cols)
        model = ModelFactory.create_model(model_type, self.config.random_state)
        pipeline = Pipeline([("preprocess", preprocessor), ("model", model)])

        # Hyperparameter tuning if enabled
        if self.config.enable_hyperparameter_tuning:
            param_grid = ModelFactory.get_hyperparameter_grid(model_type)
            if param_grid:
                self.logger.info(f"Performing hyperparameter tuning for {model_type}")

                # Use fewer splits for tuning to speed up
                tscv_tune = TimeSeriesSplit(n_splits=3)
                grid_search = GridSearchCV(
                    pipeline, param_grid, cv=tscv_tune,
                    scoring='neg_mean_squared_error',
                    n_jobs=self.config.n_jobs, verbose=1
                )
                grid_search.fit(X, y)
                pipeline = grid_search.best_estimator_
                self.logger.info(f"Best parameters for {model_type}: {grid_search.best_params_}")

        # Cross-validation evaluation
        tscv = TimeSeriesSplit(n_splits=self.config.n_splits)
        cv_scores = {'mae': [], 'rmse': [], 'r2': [], 'mape': []}
        y_true_all, y_pred_all = [], []

        for fold_idx, (train_idx, test_idx) in enumerate(tscv.split(X)):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # Fit pipeline
            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)

            # Calculate metrics (MODIFIED: Use np.sqrt for RMSE to avoid 'squared' parameter issue)
            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)
            mape = np.mean(np.abs((y_test - y_pred) / np.clip(np.abs(y_test), 1e-6, None))) * 100

            cv_scores['mae'].append(mae)
            cv_scores['rmse'].append(rmse)
            cv_scores['r2'].append(r2)
            cv_scores['mape'].append(mape)

            y_true_all.extend(y_test.tolist())
            y_pred_all.extend(y_pred.tolist())

            self.logger.info(f"{model_type} Fold {fold_idx}: MAE={mae:.4f}, RMSE={rmse:.4f}, R²={r2:.4f}")

        # Fit final model on all data
        pipeline.fit(X, y)

        # Calculate average metrics
        avg_metrics = {metric: np.mean(scores) for metric, scores in cv_scores.items()}
        std_metrics = {metric: np.std(scores) for metric, scores in cv_scores.items()}

        return {
            'pipeline': pipeline,
            'cv_metrics': cv_scores,
            'avg_metrics': avg_metrics,
            'std_metrics': std_metrics,
            'y_true': y_true_all,
            'y_pred': y_pred_all,
            'model_type': model_type
        }

    def train_models(self, df: pd.DataFrame) -> Dict[str, Dict[str, Any]]:
        """Train multiple models and compare performance"""
        self.logger.info("Starting model training...")

        # Prepare data
        df_processed, numeric_cols, categorical_cols = self.prepare_data(df)

        # Sort by date for proper time series splitting
        df_processed = df_processed.sort_values(self.config.date_col).reset_index(drop=True)

        X = df_processed.drop(columns=[self.config.target])
        y = df_processed[self.config.target].values

        self.logger.info(f"Training dataset shape: X={X.shape}, y={y.shape}")

        # Train each model
        results = {}
        for model_type in self.config.models_to_try:
            self.logger.info(f"Training {model_type} model...")

            try:
                model_results = self.train_single_model(
                    X, y, model_type, numeric_cols, categorical_cols
                )
                results[model_type] = model_results

                avg_mae = model_results['avg_metrics']['mae']
                avg_r2 = model_results['avg_metrics']['r2']
                self.logger.info(f"{model_type} completed - Average MAE: {avg_mae:.4f}, R²: {avg_r2:.4f}")

            except Exception as e:
                self.logger.error(f"Error training {model_type}: {str(e)}")
                continue

        # Select best model based on R²
        if results:
            best_model = max(results.keys(), key=lambda k: results[k]['avg_metrics']['r2'])
            self.logger.info(f"Best model: {best_model} (R² = {results[best_model]['avg_metrics']['r2']:.4f})")

            # Store best model
            self.best_model_name = best_model
            self.best_model = results[best_model]['pipeline']

        self.results = results
        return results

    def create_visualizations(self):
        """Create comprehensive visualization suite"""
        if not self.results:
            self.logger.warning("No results available for visualization")
            return

        self.logger.info("Creating visualizations...")

        # Set up plotting style
        plt.style.use('seaborn-v0_8')
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

        # 1. Model Comparison
        self._plot_model_comparison()

        # 2. Feature Importance (for best model)
        if hasattr(self, 'best_model'):
            self._plot_feature_importance()

        # 3. Residual Analysis
        self._plot_residual_analysis()

        # 4. Prediction vs Actual
        self._plot_predictions()

        # 5. Time Series Analysis
        self._plot_time_series_performance()

        self.logger.info("Visualizations completed")

    def _plot_model_comparison(self):
        """Plot model performance comparison"""
        if len(self.results) < 2:
            return

        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')

        models = list(self.results.keys())
        metrics = ['mae', 'rmse', 'r2', 'mape']
        metric_names = ['Mean Absolute Error', 'Root Mean Squared Error', 'R² Score', 'Mean Absolute Percentage Error']

        for idx, (metric, name) in enumerate(zip(metrics, metric_names)):
            ax = axes[idx//2, idx%2]

            means = [self.results[model]['avg_metrics'][metric] for model in models]
            stds = [self.results[model]['std_metrics'][metric] for model in models]

            bars = ax.bar(models, means, yerr=stds, capsize=5, alpha=0.8)
            ax.set_title(name, fontweight='bold')
            ax.set_ylabel(name)

            # Rotate x-axis labels if needed
            if len(max(models, key=len)) > 8:
                ax.tick_params(axis='x', rotation=45)

            # Add value labels on bars
            for bar, mean in zip(bars, means):
                height = bar.get_height()
                ax.annotate(f'{mean:.3f}', xy=(bar.get_x() + bar.get_width()/2, height),
                           xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

        plt.tight_layout()
        plt.savefig(self.config.plots_dir / f'model_comparison.{self.config.plot_format}',
                   dpi=self.config.plot_dpi, bbox_inches='tight')
        plt.close()

    def _plot_feature_importance(self):
        """Plot feature importance for the best model"""
        if not hasattr(self, 'best_model'):
            return

        # Extract model from pipeline
        model = self.best_model.named_steps['model']
        preprocessor = self.best_model.named_steps['preprocess']

        if not hasattr(model, 'feature_importances_'):
            return

        # Get feature names
        numeric_features = preprocessor.transformers_[0][2]
        cat_encoder = preprocessor.transformers_[1][1]
        cat_features = preprocessor.transformers_[1][2]

        if hasattr(cat_encoder, 'get_feature_names_out'):
            cat_feature_names = cat_encoder.get_feature_names_out(cat_features).tolist()
        else:
            cat_feature_names = []

        feature_names = list(numeric_features) + cat_feature_names
        importances = model.feature_importances_

        # Sort by importance
        indices = np.argsort(importances)[::-1]
        top_n = min(20, len(indices))

        plt.figure(figsize=(12, 8))
        plt.bar(range(top_n), importances[indices[:top_n]], alpha=0.8)
        plt.title(f'Top {top_n} Feature Importances - {self.best_model_name.title()}',
                 fontsize=14, fontweight='bold')
        plt.xlabel('Features')
        plt.ylabel('Importance')

        feature_labels = [feature_names[i] if i < len(feature_names) else f'Feature_{i}'
                         for i in indices[:top_n]]
        plt.xticks(range(top_n), feature_labels, rotation=90)

        plt.tight_layout()
        plt.savefig(self.config.plots_dir / f'feature_importance.{self.config.plot_format}',
                   dpi=self.config.plot_dpi, bbox_inches='tight')
        plt.close()

    def _plot_residual_analysis(self):
        """Create residual analysis plots"""
        if not hasattr(self, 'best_model_name'):
            return

        best_results = self.results[self.best_model_name]
        y_true = np.array(best_results['y_true'])
        y_pred = np.array(best_results['y_pred'])
        residuals = y_true - y_pred

        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle(f'Residual Analysis - {self.best_model_name.title()}',
                    fontsize=16, fontweight='bold')

        # 1. Residuals vs Predicted
        axes[0,0].scatter(y_pred, residuals, alpha=0.6, s=20)
        axes[0,0].axhline(y=0, color='red', linestyle='--')
        axes[0,0].set_xlabel('Predicted Values')
        axes[0,0].set_ylabel('Residuals')
        axes[0,0].set_title('Residuals vs Predicted')

        # 2. Q-Q Plot
        from scipy import stats
        stats.probplot(residuals, dist="norm", plot=axes[0,1])
        axes[0,1].set_title('Q-Q Plot (Normal Distribution)')

        # 3. Histogram of Residuals
        axes[1,0].hist(residuals, bins=30, alpha=0.7, density=True)
        axes[1,0].set_xlabel('Residuals')
        axes[1,0].set_ylabel('Density')
        axes[1,0].set_title('Distribution of Residuals')

        # Add normal curve overlay
        mu, sigma = stats.norm.fit(residuals)
        x = np.linspace(residuals.min(), residuals.max(), 100)
        axes[1,0].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', lw=2, label='Normal Fit')
        axes[1,0].legend()

        # 4. Scale-Location Plot
        sqrt_abs_resid = np.sqrt(np.abs(residuals))
        axes[1,1].scatter(y_pred, sqrt_abs_resid, alpha=0.6, s=20)
        axes[1,1].set_xlabel('Predicted Values')
        axes[1,1].set_ylabel('√|Residuals|')
        axes[1,1].set_title('Scale-Location Plot')

        plt.tight_layout()
        plt.savefig(self.config.plots_dir / f'residual_analysis.{self.config.plot_format}',
                   dpi=self.config.plot_dpi, bbox_inches='tight')
        plt.close()

    def _plot_predictions(self):
        """Plot predictions vs actual values"""
        if not hasattr(self, 'best_model_name'):
            return

        best_results = self.results[self.best_model_name]
        y_true = np.array(best_results['y_true'])
        y_pred = np.array(best_results['y_pred'])

        plt.figure(figsize=(10, 8))

        # Scatter plot
        plt.scatter(y_true, y_pred, alpha=0.6, s=20)

        # Perfect prediction line
        min_val, max_val = min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')

        # Calculate R²
        r2 = r2_score(y_true, y_pred)
        plt.text(0.05, 0.95, f'R² = {r2:.4f}', transform=plt.gca().transAxes,
                fontsize=12, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.title(f'Predictions vs Actual - {self.best_model_name.title()}',
                 fontsize=14, fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(self.config.plots_dir / f'predictions_vs_actual.{self.config.plot_format}',
                   dpi=self.config.plot_dpi, bbox_inches='tight')
        plt.close()

    def _plot_time_series_performance(self):
        """Plot model performance over time"""
        # This would require additional date tracking in cross-validation
        # For now, we'll create a placeholder
        pass

    def save_results(self):
        """Save all results and models"""
        self.logger.info("Saving results...")

        # Save model comparison metrics
        comparison_data = []
        for model_name, results in self.results.items():
            row = {'model': model_name}
            row.update(results['avg_metrics'])
            row.update({f'{k}_std': v for k, v in results['std_metrics'].items()})
            comparison_data.append(row)

        comparison_df = pd.DataFrame(comparison_data)
        comparison_df.to_csv(self.config.artifacts_dir / 'model_comparison.csv', index=False)

        # Save detailed results
        with open(self.config.artifacts_dir / 'detailed_results.json', 'w') as f:
            # Convert results to JSON-serializable format
            json_results = {}
            for model_name, results in self.results.items():
                json_results[model_name] = {
                    'cv_metrics': results['cv_metrics'],
                    'avg_metrics': results['avg_metrics'],
                    'std_metrics': results['std_metrics'],
                    'model_type': results['model_type']
                    # Note: pipeline and predictions are not JSON serializable
                }
            json.dump(json_results, f, indent=2)

        # Save best model
        if hasattr(self, 'best_model'):
            joblib.dump(self.best_model, self.config.artifacts_dir / 'best_model.joblib')

            # Save predictions from best model
            best_results = self.results[self.best_model_name]
            predictions_df = pd.DataFrame({
                'y_true': best_results['y_true'],
                'y_pred': best_results['y_pred'],
                'residuals': np.array(best_results['y_true']) - np.array(best_results['y_pred'])
            })
            predictions_df.to_csv(self.config.artifacts_dir / 'best_model_predictions.csv', index=False)

        self.logger.info(f"Results saved to {self.config.artifacts_dir}")

    def run_complete_analysis(self) -> Dict[str, Any]:
        """Run the complete ROI analysis pipeline"""
        self.logger.info("Starting complete ROI analysis...")

        try:
            # Load or generate data
            df = self.load_or_generate_data()

            # Train models
            results = self.train_models(df)

            # Create visualizations
            if self.config.save_plots:
                self.create_visualizations()

            # Save results
            self.save_results()

            # Print summary
            self._print_summary()

            return results

        except Exception as e:
            self.logger.error(f"Error in analysis pipeline: {str(e)}")
            raise

    def _print_summary(self):
        """Print analysis summary"""
        print("\n" + "="*60)
        print("ROI ENGINE ANALYSIS SUMMARY")
        print("="*60)

        if not self.results:
            print("No results available.")
            return

        # Model comparison
        print("\nModel Performance Comparison:")
        print("-" * 40)
        for model_name, results in self.results.items():
            metrics = results['avg_metrics']
            print(f"{model_name.upper():<20} | R²: {metrics['r2']:.4f} | MAE: {metrics['mae']:.4f} | RMSE: {metrics['rmse']:.4f}")

        # Best model details
        if hasattr(self, 'best_model_name'):
            print(f"\nBest Model: {self.best_model_name.upper()}")
            best_metrics = self.results[self.best_model_name]['avg_metrics']
            print(f"  R² Score: {best_metrics['r2']:.4f}")
            print(f"  Mean Absolute Error: {best_metrics['mae']:.4f}")
            print(f"  Root Mean Squared Error: {best_metrics['rmse']:.4f}")
            print(f"  Mean Absolute Percentage Error: {best_metrics['mape']:.2f}%")

        print(f"\nResults saved to: {self.config.artifacts_dir}")
        print("="*60)


# =========================
# 5) Main Execution
# =========================

def main():
    """Main execution function"""

    # Initialize configuration
    config = ROIEngineConfig(
        enable_advanced_features=True,
        enable_hyperparameter_tuning=True,
        models_to_try=['gradient_boosting', 'random_forest', 'ridge'],
        save_plots=True
    )

    # Setup logging
    logger = setup_logging(config)
    logger.info("Starting Advanced ROI Engine")

    try:
        # Initialize and run engine
        engine = AdvancedROIEngine(config, logger)
        results = engine.run_complete_analysis()

        logger.info("Analysis completed successfully")
        return results

    except Exception as e:
        logger.error(f"Analysis failed: {str(e)}")
        raise


# =========================
# 6) Execution
# =========================

if __name__ == "__main__":
    results = main()

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Fitting 3 folds for each of 24 candidates, totalling 72 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits

ROI ENGINE ANALYSIS SUMMARY

Model Performance Comparison:
----------------------------------------
GRADIENT_BOOSTING    | R²: -0.3323 | MAE: 0.0144 | RMSE: 0.0166
RANDOM_FOREST        | R²: -1.0932 | MAE: 0.0165 | RMSE: 0.0184
RIDGE                | R²: 0.0479 | MAE: 0.0109 | RMSE: 0.0124

Best Model: RIDGE
  R² Score: 0.0479
  Mean Absolute Error: 0.0109
  Root Mean Squared Error: 0.0124
  Mean Absolute Percentage Error: 13.92%

Results saved to: roi_engine_project/artifacts
